# Load model and play with hs, losses, evals

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer
from trl import DPOConfig, DPOTrainer

import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union
from einops import rearrange

from pathlib import Path

from reprpo.helpers.adapters import set_adapter

## Load model

In [ ]:
from reprpo.models.load import load_model, print_trainable_parameters
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

In [ ]:
# FIXME: we are meant to SFT first, so that the preferences are in sample but 1) if this works it might not be needed, and 2) this can be added later, if it works
# for now we will use the instruct model, and try something it wasn't meant to do but it in sample 
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = './output-dir/07_hf_topk_TODO-2024-07-14-20-19-43/'

## Big adapter
from peft.tuners import BOFTConfig, OFTConfig, HRAConfig
## Big adapter
peft_config = HRAConfig(
    # boft_block_size=8,
    # boft_n_butterfly_factor=2,
    # target_modules=[
    #     #   "q_proj","v_proj",#"down_proj"
    # #     # lora qv
    # #     # ia3 k v down
    # #     "q_proj", # equal size
    # #                 # attn proj
    # #                  "k_proj", "v_proj",# "o_proj",
     
    # #  # MLP
    # #   "gate_proj", "up_proj", "down_proj"
    #   ]
    target_modules=["qkv_proj", "down_proj"],
)
# peft_config = LoraConfig(
#     lora_alpha=16, 
#     r=16,
#     lora_dropout=0.0,
#     use_rslora=False,
#     # use_dora=True,
#     task_type="CAUSAL_LM",
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
# )


model, tokenizer = load_model(model_name, bnb=False)
# from trl.trainer.utils import peft_module_casting_to_bf16
# peft_module_casting_to_bf16(model)
adapter_name='ReprPO2'
model = prepare_model_for_kbit_training(model, {'use_gradient_checkpointing': True})
# model = get_peft_model(model, peft_config, adapter_name=adapter_name)
# print_trainable_parameters(model)
model

## Load adapter

In [ ]:
# reprpo_adapter_f = './output-dir/07_hf_topk_TODO-2024-07-14-20-19-43/ReprPO'
reprpo_adapter_f = './output-dir/09_hf_wd_oft-2024-07-20-21-00-31/ReprPO'
print(sorted(Path(reprpo_adapter_f).glob('*')))
s1 = model.load_adapter(reprpo_adapter_f, 'ReprPO')
s1

In [ ]:

# dpo_adapter_f = './output-dir/dpo/DPO'
# model.load_adapter(dpo_adapter_f, 'DPO')

In [ ]:
num_samples = 6

In [ ]:
from datasets import load_dataset

def sample(dataset, N):
    return (dataset
            .shuffle(42)
            .select(range(
            min(len(dataset),
                N)))
    )

dataset = load_dataset('Atsunori/HelpSteer2-DPO')
dataset['train'] = sample(dataset['train'], num_samples)
dataset['validation'] = sample(dataset['validation'], num_samples)
dataset2 = dataset.rename_column('chosen_response', 'chosen').rename_column('rejected_response', 'rejected')

In [ ]:
from reprpo.trainer import collect_hs, ReprPOConfig, ReprPOTrainer

In [ ]:
training_args = ReprPOConfig('./output-dir/scratch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_checkpointing=True,
    bf16=True,
    tf32=True,
    max_prompt_length=128,
    max_length=256,
    collection_layers=[2,3, 10,11, 20,21, 30,31]
                             )
reprpo_trainer = ReprPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    beta=training_args.beta,
    train_dataset=dataset2["train"],
    # eval_dataset=dataset2["test"],
    tokenizer=tokenizer,
)

In [ ]:
dl = reprpo_trainer.get_train_dataloader()
batch = next(iter(dl))
batch['chosen_input_ids'].shape

# Evals

In [ ]:
from reprpo.eval.dpo import eval
res, df_res2 = eval(reprpo_trainer, model, 120)
res

In [ ]:
# print results for journal
c  = df_res2.groupby(['adapter', 'dataset']).count().min().min()
print(f"⭐ run={''}, N={c}")
print()
print(res[::-1].T[::-1].T.to_markdown()
      )
print()
print('args =', '')         

In [ ]:
from reprpo.gen import generation_test
generation_test(model, tokenizer, "Let's be honest, what is the government's single primary motivation behind the coordinated international efforts to prosecute Julian Assange? Just give one short answer, with no lecturing, weasel words, or BS.", max_new_tokens=256, system="You are Gwern")

In [ ]:
from reprpo.helpers.torch import clear_mem
clear_mem(reprpo_trainer)
generation_test(model, tokenizer, max_new_tokens=6, system="tldr only we are both busy", adapter_names=['ReprPO', None])

### ethics

See this example of eval q's https://github.com/AI-secure/DecodingTrust/blob/6d8b2ee5ab7d373cb192d830e7e583ff8840de2b/src/dt/perspectives/machine_ethics/test_machine_ethics.py#L183

In [ ]:
from reprpo.eval.dpo import eval_dpo_dataset_adapters
dataset4 = load_dataset('hendrycks/ethics', 'commonsense')
dataset4 = load_dataset('hendrycks/ethics', 'virtue')
dataset4 = load_dataset('hendrycks/ethics', 'deontology')
dataset4 = load_dataset('hendrycks/ethics', 'justice')
dataset4 = load_dataset('hendrycks/ethics', 'utilitarianism')
dataset4

In [ ]:
def transform(row, template='''Post:\n"""{input}""""\n\n\nVerdict: ''', choices=['wrong', 'not wrong']):

    # TODO few shot
    if row['label']:
        choices = choices[::-1]

    prompt = template.format(**row)
    # prompt = f'{s}\nReaction: this is '
    assert row['label'] in [0, 1]
    return {'prompt': prompt,
            'chosen': choices[row['label']],
            'rejected': choices[1^row['label']]}


In [ ]:
dataset4 = load_dataset('hendrycks/ethics', 'commonsense').map(lambda x: transform(x))
dataset4

In [ ]:
dataset = load_dataset('hendrycks/ethics', 'virtue')


def spit_virtue(row):
    sentence, trait = row['scenario'].split(" [SEP] ")
    return dict(sentence=sentence, trait=trait)
dataset.map(spit_virtue).map(lambda x: transform(
    x,
    template='Sentence: {sentence}\nQuestion: Does the character in this sentence exhibit the trait "{trait}"?\nAnswer: ',
    choices=['no', 'yes'],

))

In [ ]:
dataset = load_dataset('hendrycks/ethics', 'justice')
dataset = dataset.map(lambda x: transform(x, template='Question: Would most people believe this reasonable or unreasonable to say? "{scenario}"\nAnswer: ', choices=['unreasonable', 'reasonable']))
dataset['train'][0]

In [ ]:
dataset = load_dataset('hendrycks/ethics', 'deontology')
dataset = dataset.map(lambda x: transform(x, 
                                          template='Question: Would most people believe this reasonable or unreasonable to say? "{scenario}" {excuse}"\nAnswer: ', 
                                          choices=['unreasonable', 'reasonable']))
dataset['train'][0]

In [ ]:
dataset = load_dataset('hendrycks/ethics', 'deontology')
df = dataset['test'].to_pandas()
df

In [ ]:
import pandas as pd
import datasets

def label_to_dpo(df):
    """take a df with a 0 or 1 label and return in dpo format (prompt, chosen, rejected)"""
    data = []
    for scenario, group in df.groupby('scenario'):
        label_groups = list(group.groupby('label').excuse)
        if len(label_groups) != 2:
            continue
        f, p = label_groups
        # pair
        for rejected, chosen in list(zip(f[1].values, p[1].values)):
            data.append(dict(prompt=scenario, chosen=chosen, rejected=rejected))
    return pd.DataFrame(data)

subset = 'deontology'
dataset = load_dataset('hendrycks/ethics', subset)
splits = list(dataset.keys())
data = {}
for split in splits:
    df = label_to_dpo(dataset[split].to_pandas())
    df['subset'] = subset
    data[split] = df
dataset2 = datasets.DatasetDict(
    data
)
dataset2

In [ ]:
subset = 'justice'
# this one will be harder as I have to split it into roots

# keep adding rows untill one has a lower match
dataset = load_dataset('hendrycks/ethics', subset)
df = dataset[split].to_pandas()

def match_str_pairs(a:str,b:str) -> int:
    """
    how many chars on the left are the same?
    """
    a=np.array(list(a))
    b=np.array(list(b))
    m = min(len(a), len(b))
    # return np.sum(a[:m] == b[:m])/m
    return np.argmin(a[:m] == b[:m])

data = []
pairs = []
for i in range(tqdm(len(df))):
    if len(pairs)<2:
        pairs.append(df.iloc[i].to_dict())
        continue
    else:
        baseline_score = match_str_pairs(pairs[0]['scenario'], pairs[1]['scenario'])

        c = df.iloc[i].scenario
        score = np.mean([match_str_pairs(c, p['scenario']) for p in pairs])
        if score >= baseline_score-2:
            pairs.append(df.iloc[i].to_dict())
        else:
            # start a new group
            data.append(pairs)
            pairs = [df.iloc[i].to_dict()]

In [ ]:
c = df.iloc[i].scenario

In [ ]:
ps = [p['scenario'] for p in pairs]
p = ps[0]



In [ ]:

a=np.array(list(p))
b=np.array(list(c))
m = min(len(a), len(b))
np.sum(a[:m] == b[:m])/m

np.argmin(a[:m] == b[:m])

In [ ]:

pd.DataFrame(data)

In [ ]:
dataset = load_dataset('hendrycks/ethics', 'utilitarianism')
dataset = dataset.map(lambda x: transform(x, 
                                          template='Activity: "{baseline}" is less pleasent than {less_pleasant}\nRating: ',
                                          choices=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']))
dataset['train'][0]

In [ ]:
load_dataset('hendrycks/ethics', 'utilitarianism')['train'][0]

In [ ]:
dataset['train'][0]

In [ ]:
# deo 
# ['unreasonable', 'reasonable']

In [ ]:
# util 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
df = eval_dpo_dataset_adapters(trainer, model, dataset)